In [1]:
pip install twstock

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.1/626.1 kB 12.3 MB/s eta 0:00:00


In [2]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import requests, json
from twstock import Stock
from tqdm import tqdm

In [3]:
mth = datetime.now().month
mth

7

# 股價

In [4]:
stock_no = '0050'
stock = Stock(stock_no)


df = pd.DataFrame()
month = range(1,13)
for M in tqdm(month,desc="Downloading..."):
  if M == datetime.now().month:
    break
  else:
    data = stock.fetch_from(2024,M)
    df_new = pd.DataFrame(data)
    df = pd.concat([df, df_new])

df

Downloading...:  50%|█████     | 6/12 [00:29<00:29,  4.94s/it]


,date,capacity,turnover,open,high,low,close,change,transaction
0,2024-01-02,6283610,848793104,135.60,135.95,134.65,134.90,-0.55,10013
1,2024-01-03,14783894,1963126583,133.70,133.90,132.30,132.55,-2.35,25258
2,2024-01-04,4838660,641202865,132.55,132.75,132.30,132.50,-0.05,7094
3,2024-01-05,4566320,604475455,132.55,132.95,132.10,132.15,-0.35,12744
4,2024-01-08,9657556,1285524342,133.00,133.60,132.75,132.75,0.60,9876
...,...,...,...,...,...,...,...,...,...
31,2024-07-17,15694924,3059326947,196.40,196.50,193.60,194.10,-2.75,31123
32,2024-07-18,19796818,3754498885,188.60,190.90,188.20,190.60,-3.50,39463
33,2024-07-19,18436231,3456363627,188.75,189.00,186.20,186.25,-4.35,43183
34,2024-07-22,23474914,4268806758,185.40,185.40,180.05,180.70,-5.55,48514


date
datetime.datetime 格式之時間，例如 datetime.datetime(2017, 6, 12, 0, 0)。

capacity
總成交股數 (單位: 股)。

turnover
總成交金額 (單位: 新台幣/元)。

open
開盤價。

high
盤中最高價

low
盤中最低價。

close
收盤價。

change
漲跌價差。

transaction
成交筆數。

In [5]:
df = df[[
    'date','capacity', 'turnover', 'change', 'transaction', 'open', 'high', 'low', 'close'
]]
df.head()

,date,capacity,turnover,change,transaction,open,high,low,close
0,2024-01-02,6283610,848793104,-0.55,10013,135.60,135.95,134.65,134.90
1,2024-01-03,14783894,1963126583,-2.35,25258,133.70,133.90,132.30,132.55
2,2024-01-04,4838660,641202865,-0.05,7094,132.55,132.75,132.30,132.50
3,2024-01-05,4566320,604475455,-0.35,12744,132.55,132.95,132.10,132.15
4,2024-01-08,9657556,1285524342,0.60,9876,133.00,133.60,132.75,132.75


In [6]:
df.to_csv("/content/0050.csv",index=False)

# 成分

In [ ]:
def load_data():
    url = 'https://zh.wikipedia.org/wiki/%E8%87%BA%E7%81%A350%E6%8C%87%E6%95%B8'
    html = pd.read_html(url, header = 0)
    df = html[0]
    return df

df = load_data()
df1 = df[['股票代號','名稱']]
df2 = df[['股票代號.1','名稱.1']]
df2.columns = ['股票代號','名稱']
df = pd.concat([df1, df2]).reset_index(drop=True)
df = df['股票代號'].str.split('：', n=1, expand=True)
df = df[1].to_list()
df

['1101',
 '1216',
 '1301',
 '1303',
 '1326',
 '1590',
 '2002',
 '2207',
 '2301',
 '2303',
 '2308',
 '2317',
 '2327',
 '2330',
 '2345',
 '2357',
 '2379',
 '2382',
 '2395',
 '2408',
 '2412',
 '2454',
 '2603',
 '2880',
 '2881',
 '2882',
 '2883',
 '2884',
 '2885',
 '2886',
 '2887',
 '2890',
 '2891',
 '2892',
 '2912',
 '3008',
 '3017',
 '3034',
 '3037',
 '3045',
 '3231',
 '3661',
 '3711',
 '4904',
 '4938',
 '5871',
 '5876',
 '5880',
 '6505',
 '6669']

## Download all

In [ ]:
stock_price = []
for stock_name in tqdm(df, desc="Fetching stock prices"):
  stock = Stock(stock_name)
  data = stock.fetch_from(2024,7)

  single_stock = pd.DataFrame(data)
  single_stock['stock_name'] = stock_name
  stock_name_col = single_stock.pop('stock_name')
  single_stock.insert(0, 'stock_name', stock_name_col)

  stock_price.append(single_stock)

Fetching stock prices: 100%|██████████| 50/50 [04:59<00:00,  5.99s/it]


In [ ]:
all_stock = pd.concat(stock_price, axis=0)
all_stock.head(20)

,stock_name,date,capacity,turnover,open,high,low,close,change,transaction
0,1101,2024-07-01,19316005,649726579,33.60,33.85,33.55,33.55,0.00,9327
1,1101,2024-07-02,20261082,683901390,33.50,34.00,33.40,33.70,0.15,8563
2,1101,2024-07-03,14603385,494136634,33.80,34.00,33.60,34.00,0.30,7110
3,1101,2024-07-04,17205995,585985695,34.00,34.20,33.95,34.15,0.15,7590
4,1101,2024-07-05,16451159,565109024,34.20,34.50,34.10,34.45,0.30,7431
5,1101,2024-07-08,18504594,639511550,34.55,34.65,34.40,34.60,0.15,10215
0,1216,2024-07-01,4114955,334846625,81.30,82.00,80.90,81.50,0.20,3882
1,1216,2024-07-02,8141817,665685295,81.00,82.30,80.90,82.00,0.50,3421
2,1216,2024-07-03,10161018,836616886,82.00,82.90,81.70,82.00,0.00,5476
3,1216,2024-07-04,10489153,881313062,82.90,84.70,82.60,84.20,2.20,7300
